##### En esta fase, se desarrolla un análisis preliminar orientado a describir el conjunto de datos y evaluar su consistencia. Esta revisión permite identificar el tipo de variables disponibles y posibles incidencias que puedan afectar al análisis. Por el momento, no se ejecutan correcciones ni transformaciones, ya que las tareas de limpieza y transformación se abordarán en etapas posteriores.

### Importación de librerías

In [2]:
# Tratamiento de datos.

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# Visualizaciones.

import matplotlib.pyplot as plt
import seaborn as sns

### Carga de datos `fraudTrain.csv`

In [3]:
df_fraud = pd.read_csv("../data/1.raw/fraudTrain.csv", index_col= 0)

### Descripción de variables 

index: Identificador único de cada fila

trans_date_trans_time: Fecha y hora de la transacción

cc_num: Número de tarjeta de crédito del cliente

merchant: Nombre del comercio

category: Categoría del comercio

amt: Importe de la transacción

first: Nombre del titular de la tarjeta

last: Apellidos del titular de la tarjeta

gender: Género del titular de la tarjeta

street: Dirección (calle) del titular de la tarjeta

city: Ciudad del titular de la tarjeta

state: Estado del titular de la tarjeta

zip: Código postal del titular de la tarjeta

lat: Latitud de la ubicación del titular de la tarjeta

long: Longitud de la ubicación del titular de la tarjeta

city_pop: Población de la ciudad del titular

job: Profesión/ocupación del titular

dob: Fecha de nacimiento del titular

trans_num: Número/identificador de la transacción

unix_time: Marca de tiempo UNIX de la transacción

merch_lat: Latitud de la ubicación del comercio

merch_long: Longitud de la ubicación del comercio

is_fraud: Indicador de fraude (variable objetivo)

index: ID interno único por registro (no aporta información de negocio).

trans_date_trans_time: Timestamp de la compra; útil para patrones horarios, días de la semana y estacionalidad.

cc_num: Identificador de tarjeta/cliente (dato sensible; recomendable anonimizar o eliminar antes de publicar).

merchant: Comercio donde se realiza la transacción; permite análisis por comercio y detección de comercios de riesgo.

category: Tipo de comercio (p. ej., gasolina, alimentación); útil para segmentación de patrones.

amt: Importe de la compra; variable clave para outliers y detección de anomalías.

first, last: Identificadores personales del titular (PII; recomendable eliminar/anonimizar).

gender: Género del titular; puede usarse en segmentación, con cautela por sesgos.

street, city, state, zip: Localización del titular (PII); útil para análisis geográfico y distancia, pero conviene anonimizar para publicación.

lat, long: Coordenadas del titular; permiten cálculos de distancia y patrones espaciales.

city_pop: Tamaño de la ciudad del titular; proxy de densidad/urbanidad.

job: Ocupación del titular; variable categórica para perfilado (con posible ruido).

dob: Fecha de nacimiento; permite derivar edad (dato sensible, mejor derivar edad y descartar dob).

trans_num: Identificador único de transacción; útil para trazabilidad y control de duplicados.

unix_time: Timestamp en formato UNIX; alternativa para ordenar/transformar fechas.

merch_lat, merch_long: Coordenadas del comercio; permiten calcular distancia titular–comercio y detectar ubicaciones atípicas.

is_fraud: Etiqueta objetivo (1 = fraude, 0 = no fraude).

### Análisis preliminar de `fraudTrain.csv`

Se visualiza una muestra inicial del dataset para confirmar la correcta importación de los datos.

In [4]:
df_fraud.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-12-13 10:58:20,4681699462969,fraud_Kling Inc,gas_transport,92.53,Joseph,Gonzalez,M,319 Wendy Fort Suite 179,Murfreesboro,TN,37132,35.8596,-86.4210,158701,"Journalist, newspaper",1978-03-06,1e9a5baf2d98ea7a21158a8adb1bcbb7,1355396300,35.233931,-86.053992,0
1,2019-06-22 12:28:00,4364010865167176,fraud_Predovic Inc,shopping_net,5.79,Gary,Martinez,M,03512 Jackson Ports,Reno,NV,89512,39.5483,-119.7957,276896,Immunologist,1997-03-12,bf07aaaa3b2f4775dc2f5d3c222bd986,1340368080,40.432188,-119.956735,0
2,2019-04-28 19:43:17,4040099974063068803,"fraud_Turner, Ziemann and Lehner",food_dining,55.25,Jeffrey,Lewis,M,24255 Bryan Square,Palermo,ND,58769,48.3396,-102.2400,229,Administrator,1983-03-20,4a587132a79ebc2091832a926d096977,1335642197,49.123042,-102.129622,0
3,2019-07-16 08:08:34,4555104582813474,fraud_Gerlach Inc,shopping_net,82.60,Chris,Daniel,M,025 White Fork Apt. 633,Rock Glen,PA,18246,40.9540,-76.1747,143,Health and safety adviser,1982-02-19,a19a0d24d6fab24f5a4cd98ce2d8af43,1342426114,41.787515,-75.749170,0
4,2019-07-23 03:29:17,30026790933302,"fraud_Rohan, White and Aufderhar",misc_net,104.96,John,Peters,M,555 Michael Burgs,Mayersville,MS,39113,32.9013,-91.0286,595,Technical brewer,1979-09-03,574010e975db3d75163768d20d1fec0c,1343014157,32.201658,-90.782759,0


Se muestran los últimos registros para confirmar que el dataset se ha importado correctamente y que la lectura del archivo se ha completado sin incidencias.

In [5]:
df_fraud.tail()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
389995,2019-06-20 15:07:33,371284100299909,"fraud_Baumbach, Strosin and Nicolas",shopping_pos,8.23,Hannah,Thomas,F,1004 Willis Pass,Hedley,TX,79237,34.8698,-100.6806,513,Early years teacher,1976-05-24,7fa24d3d1c075467ba65e1a092c163f7,1340204853,34.374344,-100.005574,0
389996,2019-04-25 12:02:16,3592325941359225,fraud_Erdman-Ebert,personal_care,9.78,Ashley,Robinson,F,1007 Colton Forks,Hopewell,VA,23860,37.2876,-77.2950,31970,Purchasing manager,1935-08-15,d5c7836b37a4ed28c1304eb847f7dca7,1335355336,36.750143,-78.190098,0
389997,2019-10-25 16:49:47,5559857416065248,"fraud_O'Connell, Botsford and Hand",home,26.62,Jack,Hill,M,5916 Susan Bridge Apt. 939,Grenada,CA,96038,41.6125,-122.5258,589,Systems analyst,1945-12-21,63f49489d1ff4d4701e756ba2837dc05,1351183787,41.338409,-123.105357,0
389998,2019-01-15 20:24:39,3564182536169293,"fraud_Witting, Beer and Ernser",home,3.70,Brenda,Johnson,F,56160 Nicholas Isle,Norwich,OH,43767,39.9934,-81.8024,1443,Research scientist (medical),1962-03-04,9a08350d00bb9bf65569c42e7f20b3c9,1326659079,39.511339,-82.754286,0
389999,2019-10-05 19:15:19,377113842678100,fraud_Kassulke Inc,entertainment,14.30,Billy,Gallagher,M,673 Delgado Burg,Greenwich,NJ,8323,39.4055,-75.3209,804,Insurance risk surveyor,1965-03-25,a7b2be172f556633c1d117bd70203af7,1349464519,40.065891,-75.417425,0


### Dimensión del dataset

Se revisan las dimensiones del conjunto de datos para conocer su estructura general.

In [6]:
print(f"El número de filas es {df_fraud.shape[0]} y el número de columnas es {df_fraud.shape[1]} ")

El número de filas es 390000 y el número de columnas es 22 


### Variables del conjunto de datos

Se inspeccionan los nombres de las columnas para comprender la estructura del dataset.

In [7]:
df_fraud.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

### Información del dataset

Se revisa la información del dataframe para detectar nulos y asegurar que los tipos de datos se presentan en los esperados.

In [8]:
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390000 entries, 0 to 389999
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  390000 non-null  object 
 1   cc_num                 390000 non-null  int64  
 2   merchant               390000 non-null  object 
 3   category               390000 non-null  object 
 4   amt                    390000 non-null  float64
 5   first                  390000 non-null  object 
 6   last                   390000 non-null  object 
 7   gender                 390000 non-null  object 
 8   street                 390000 non-null  object 
 9   city                   390000 non-null  object 
 10  state                  390000 non-null  object 
 11  zip                    390000 non-null  int64  
 12  lat                    390000 non-null  float64
 13  long                   390000 non-null  float64
 14  city_pop               390000 non-null  i

**Observaciones:**

El análisis confirma que no existen valores nulos en ninguna variable.

La variable `trans_date_trans_time`, que registra la fecha y hora de la transacción, está almacenada como tipo **object** cuando debería estar en formato **datetime**.

La variable `dob`, corresponde a la fecha de nacimiento del titular, también aparece como tipo **object** y debe convertirse a **datetime**.

### Registros duplicados

Se revisa la presencia de filas duplicadas para comprobar la consistencia e integridad del conjunto de datos.

In [33]:
df_fraud.duplicated().sum()

np.int64(0)

### Exploración de variables temporales

Se analizan las variables temporales asociadas a la transacción y al titular de la tarjeta para verificar su formato, rango y consistencia.

In [9]:
df_fraud['trans_date_trans_time'] = pd.to_datetime(df_fraud['trans_date_trans_time'])

print(f"Los valores únicos de los años para la columna {'trans_date_trans_time'} son: {df_fraud['trans_date_trans_time'].dt.year.unique()}")

Los valores únicos de los años para la columna trans_date_trans_time son: [2019 2020]


Las transacciones del dataset se concentran en el periodo **2019–2020**, sin registros fuera de ese rango.

In [10]:
df_fraud["dob"] = pd.to_datetime(df_fraud["dob"])

años = np.sort(df_fraud["dob"].dt.year.unique())

print(f"Los valores únicos de los años para la columna dob son: {años}")

Los valores únicos de los años para la columna dob son: [1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937
 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951
 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965
 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979
 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993
 1994 1995 1996 1997 1998 1999 2000 2001 2003 2004 2005]


Las fechas de nacimiento abarcan desde **1924** hasta **2005**, reflejando un rango amplio de edades entre los titulares.

### Registros duplicados

Se identifican los registros duplicados para evaluar la integridad del conjunto de datos.

In [11]:
df_fraud.duplicated().sum()

np.int64(0)

Tras la revisión del conjunto de datos, no se identifican registros duplicados en ninguna variable.

### Análisis de variables

Se analiza la estructura de las variables para identificar su tipología y su distribución.

#### Variables numéricas

In [12]:
columns_num_fraud = df_fraud.select_dtypes(include='number').columns

columns_num_fraud

Index(['cc_num', 'amt', 'zip', 'lat', 'long', 'city_pop', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

Se identifican 10 variables numéricas en el dataset, relacionadas con el importe de la transacción, ubicación del cliente y del comercio, contexto demográfico y tiempo, además de la variable objetivo `is_fraud`.

In [13]:
df_fraud.describe(include='number').T

,count,mean,std,min,25%,50%,75%,max
cc_num,390000.0,4.150521e+17,1.305650e+18,6.041621e+10,1.800429e+14,3.521417e+15,4.642255e+15,4.992346e+18
amt,390000.0,7.016052e+01,1.546312e+02,1.000000e+00,9.670000e+00,4.762000e+01,8.287000e+01,2.508694e+04
zip,390000.0,4.884269e+04,2.692498e+04,1.257000e+03,2.623700e+04,4.817400e+04,7.204200e+04,9.978300e+04
lat,390000.0,3.854814e+01,5.078996e+00,2.002710e+01,3.462050e+01,3.937160e+01,4.194040e+01,6.669330e+01
long,390000.0,-9.026905e+01,1.378817e+01,-1.656723e+02,-9.680940e+01,-8.747690e+01,-8.015800e+01,-6.795030e+01
city_pop,390000.0,8.846263e+04,3.004152e+05,2.300000e+01,7.430000e+02,2.443000e+03,2.032800e+04,2.906700e+06
unix_time,390000.0,1.349250e+09,1.283426e+07,1.325376e+09,1.338766e+09,1.349250e+09,1.359373e+09,1.371817e+09
merch_lat,390000.0,3.854676e+01,5.112171e+00,1.903124e+01,3.473474e+01,3.937620e+01,4.196248e+01,6.739702e+01
merch_long,390000.0,-9.026815e+01,1.380127e+01,-1.666662e+02,-9.694200e+01,-8.747752e+01,-8.025377e+01,-6.695600e+01
is_fraud,390000.0,5.789744e-03,7.586987e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


**Observaciones:**

El resumen estadístico de las variables numéricas muestra que el conjunto de datos contiene **390.000** registros.

`cc_num`: actúa como un identificador de tarjeta, su media y su desviación no son interpretables como magnitudes, por eso es preferible tratarla como un ID y no como variable numérica de análisis.

`amt`: presenta una distribución claramente asimétrica, con valores máximos muy elevados de hasta **25.086,94** en comparación con el resto de transacciones, lo que sugiere presencia de outliers.

`zip`: funciona como un código de identificación geográfica, ya que lo único que muestra es el código postal del titular de la tarjeta, y por esta razón sus valores no representan una magnitud numérica interpretable, en consecuencia no debería tratarse como variable numérica, sino como una variable categórica.

`lat` , `long` , `merch_lat` y `merch_long`: presentan estadísticos muy similares, lo que permite incorporar variables espaciales como distancia o discrepancia geográfica.

`city_pop`: muestra una fuerte variabilidad, indicando que las transacciones abarcan desde localidades pequeñas hasta grandes áreas urbanas, conviene evaluar transformaciones o segmentaciones por tamaño de ciudad para mejorar la interpretación.

`is_fraud`: presenta tasa de fraude aproximada del **0,58%**. Esto confirma un desbalanceo importante de la clase objetivo, aspecto importante a considerar en el análisis.

In [14]:
for col in columns_num_fraud:
    
    print(f'La columna {col} tiene un total de {df_fraud[col].nunique()} valores únicos')

    display(df_fraud[col].value_counts().head(10))

La columna cc_num tiene un total de 982 valores únicos


cc_num
213141712584544     979
4716561796955522    975
6534628260579800    970
6011438889172900    959
6538891242532018    956
3545109339866548    956
180048185037117     953
571365235126        953
213112402583773     951
3560797065840735    950
Name: count, dtype: int64

La columna amt tiene un total de 33425 valores únicos


amt
1.25    175
1.09    172
1.23    170
1.70    170
1.04    169
1.05    166
1.28    166
1.19    162
1.14    161
1.20    161
Name: count, dtype: int64

La columna zip tiene un total de 969 valores únicos


...

merch_lat
41.271468    4
40.946823    3
40.954851    3
38.196353    3
40.015085    3
41.659134    3
41.803552    3
43.733171    3
40.774654    3
41.404202    3
Name: count, dtype: int64

La columna merch_long tiene un total de 388102 valores únicos


merch_long
-79.393911    3
-80.484547    3
-76.326526    3
-83.498739    3
-82.283919    3
-79.322210    3
-79.394018    3
-88.417176    3
-86.154120    2
-83.762112    2
Name: count, dtype: int64

La columna is_fraud tiene un total de 2 valores únicos


is_fraud
0    387742
1      2258
Name: count, dtype: int64

**Observaciones:**

El recuento de valores únicos muestra que el dataset registra muchas transacciones repetidas sobre un conjunto limitado de **982** tarjetas y **969** códigos postales, lo que indica una repetición de operaciones asociadas a los mismos titulares y zonas geográficas.

Las variables de localización del titular y la población de la ciudad también presentan un número reducido de valores distintos, indicando que una parte relevante de las transacciones se concentra en ubicaciones y ciudades de tamaño similares.

En cambio, `unix_time`, `merch_lat` y `merch_long` presentan una cardinalidad muy alta, casi un valor distinto por registro, lo que sugiere variación temporal y geográfica elevada a nivel de transacción y lo que facilita análisis temporales y por geoespaciales a un nivel más detallado.

Por último, `is_fraud` confirma un marcado desbalanceo de clases, con **2.258** fraudes frente a **387.742** transacciones no fraudulentas, aspecto relevante para el análisis y cualquier enfoque predictivo posterior.

In [15]:
df_fraud.median(numeric_only=True).round(2)

cc_num        3.521417e+15
amt           4.762000e+01
zip           4.817400e+04
lat           3.937000e+01
long         -8.748000e+01
city_pop      2.443000e+03
unix_time     1.349250e+09
merch_lat     3.938000e+01
merch_long   -8.748000e+01
is_fraud      0.000000e+00
dtype: float64

**Observaciones:**

`amt` presenta un valor típico cercano a **47,62** y `city_pop` alrededor de **2.443**, lo que sugiere que una parte importante de las transacciones proviene de ciudades relativamente pequeñas. La variable objetivo `is_fraud` mantiene un valor central de **0**, coherente con la baja proporción de fraudes en el dataset.

### Variables categoricas

In [16]:
columns_cate_fraud = df_fraud.select_dtypes(include= ['category', 'object']).columns

columns_cate_fraud

Index(['merchant', 'category', 'first', 'last', 'gender', 'street', 'city',
       'state', 'job', 'trans_num'],
      dtype='object')

Se identifican 10 variables categóricas que describen el tipo de comercio y su categoría, datos del titular y campos de localización, además del identificador único de cada transacción.

In [17]:
df_fraud.describe( include=['category', 'object']).T

,count,unique,top,freq
merchant,390000,693,fraud_Kilback LLC,1369
category,390000,14,gas_transport,39746
first,390000,352,Christopher,8060
last,390000,481,Smith,8690
gender,390000,2,F,213438
street,390000,982,742 Oneill Shore,979
city,390000,893,Birmingham,1704
state,390000,51,TX,28350
job,390000,494,Film/video editor,2922
trans_num,390000,390000,75d0223eec8153ab4100005255ecbb41,1


**Observaciones:**

En el análisis de variables categóricas se distinguen dos perfiles, por un lado `category` y `gender` que presentan un número reducido de categorías, mientras que `merchant`, `street`, `city` y `job` contienen una gran variedad de valores diferentes. Por último, se observa que `trans_num` es un identificador único por registro y no aporta valor como categoría para segmentación, por lo que debe utilizarse únicamente como clave de transacción.

In [18]:
for col in columns_cate_fraud:
    
    print(f"La columna {col} tiene un total de {df_fraud[col].nunique()} valores únicos")

    display(df_fraud[col].value_counts().head(10)) 

La columna merchant tiene un total de 693 valores únicos


merchant
fraud_Kilback LLC                   1369
fraud_Schumm PLC                    1078
fraud_Cormier LLC                   1066
fraud_Boyer PLC                     1061
fraud_Kuhn LLC                      1043
fraud_Dickinson Ltd                 1034
fraud_Jenkins, Hauck and Friesen     859
fraud_Rodriguez Group                854
fraud_Corwin-Collins                 839
fraud_Bartoletti-Wunsch              838
Name: count, dtype: int64

La columna category tiene un total de 14 valores únicos


category
gas_transport     39746
grocery_pos       37220
home              37193
shopping_pos      34807
kids_pets         33837
shopping_net      29173
entertainment     28296
food_dining       27548
personal_care     27266
health_fitness    26029
Name: count, dtype: int64

La columna first tiene un total de 352 valores únicos


...

state
TX    28350
NY    25000
PA    24099
CA    17103
OH    14003
MI    13773
IL    12965
FL    12777
AL    12276
MO    11552
Name: count, dtype: int64

La columna job tiene un total de 494 valores únicos


job
Film/video editor             2922
Exhibition designer           2748
Naval architect               2598
Surveyor, land/geomatics      2517
Designer, ceramics/pottery    2496
Materials engineer            2465
Financial adviser             2302
IT trainer                    2297
Systems developer             2269
Environmental consultant      2222
Name: count, dtype: int64

La columna trans_num tiene un total de 390000 valores únicos


trans_num
75d0223eec8153ab4100005255ecbb41    1
5476548a9f32f3e5c641a54b82bcaccf    1
74fc8fff43c567d158aa68abd2e45c75    1
67659dee7d9106bc397be3af3f0a6196    1
bc95ff87f2c930a4b59acfabb00118c2    1
840aad54961bd5e97f9120e442f3d7ea    1
bacfb987316635a61b3c4a80e83ad068    1
25a87830420bcffc64b2d5a098a101db    1
e33daa9abd00b92cf5a5cf951d7c210c    1
fea984949481c17d6f67469267b59adf    1
Name: count, dtype: int64

**Observaciones:**

Los resultados de los valores únicos evidencian que `category` está bien estructurada para análisis por tipo de gasto y que `gender` aporta una segmentación básica que se limita a dos modalidades. 

Sin embargo, campos como `merchant`, `street`, `city` y `job` presentan una dispersión considerable de valores, por lo que conviene realizar agrupaciones para su analisis, y así evitar fragmentación en gráficos y métricas. 

Por otro lado, `trans_num` se comporta como clave primaria al no repetir ningún valor, lo indica que su función es puramente identificativa y no descriptiva.

### Carga de datos `IRSIncomeByZipCode.xlsx`


In [19]:
df_income = pd.read_excel("../data/1.raw/IRSIncomeByZipCode.xlsx", index_col= 0)

### Descripción de variables 

STATE: Abreviatura de dos letras del estado en el que se encuentra el código postal. (String)
ZIPCODE: Código postal (ZIP) de EE. UU. de cinco dígitos. (Integer)
Number of returns: Número total de declaraciones de impuestos presentadas en el código postal. (Integer)
Adjusted gross income (AGI): Importe total del ingreso bruto ajustado (AGI) declarado en el código postal. (Integer)
Avg AGI: Promedio del ingreso bruto ajustado (AGI) declarado en el código postal. (Integer)
Number of returns with total income: Total de declaraciones con ingresos totales informados en el código postal. (Integer)
Total income amount: Importe total de ingresos declarado en el código postal. (Integer)
Avg total income: Promedio del ingreso total declarado en el código postal. (Integer)
Number of returns with taxable income: Total de declaraciones con ingreso imponible informado en el código postal. (Integer)
Taxable income amount: Importe total del ingreso imponible declarado en el código postal. (Integer)
Avg taxable income: Promedio del ingreso imponible declarado en el código postal. (Integer)

### Análisis preliminar de `IRSIncomeByZipCode.xlsx`

Se muestra una vista previa del segundo dataset con el fin de validar su correcta importación.

In [20]:
df_income.head()

,STATE,ZIPCODE,Number of returns,Adjusted gross income (AGI),Avg AGI,Number of returns with total income,Total income amount,Avg total income,Number of returns with taxable income,Taxable income amount,Avg taxable income
index,,,,,,,,,,,
0,AL,0,2022380,105089761,51.963410,2022380,106420533,52.621433,1468370,67850874,46.208295
1,AL,35004,4930,255534,51.832454,4930,258024,52.337525,4020,163859,40.760945
2,AL,35005,3300,128387,38.905152,3300,129390,39.209091,2440,70760,29.000000
3,AL,35006,1230,58302,47.400000,1230,58585,47.630081,940,36341,38.660638
4,AL,35007,11990,643708,53.687073,11990,651350,54.324437,9280,414878,44.706681


Se examinan los registros finales para comprobar que la lectura del dataset se ha realizado de principio a fin de forma correcta.

In [21]:
df_income.tail()

,STATE,ZIPCODE,Number of returns,Adjusted gross income (AGI),Avg AGI,Number of returns with total income,Total income amount,Avg total income,Number of returns with taxable income,Taxable income amount,Avg taxable income
index,,,,,,,,,,,
27785,WY,83126,150,8176,54.506667,150,8319,55.460000,110,4543,41.300000
27786,WY,83127,1400,87014,62.152857,1400,88398,63.141429,1060,58258,54.960377
27787,WY,83128,860,62354,72.504651,860,63379,73.696512,680,45017,66.201471
27788,WY,83414,200,21393,106.965000,200,22319,111.595000,170,15853,93.252941
27789,WY,99999,24460,2117021,86.550327,24460,2154270,88.073181,19990,1656600,82.871436


### Dimensión del dataset

Se comprueban las dimensiones del conjunto de datos para identificar su tamaño y estructura.

In [22]:
print(f"El número de filas es {df_income.shape[0]} y el número de columnas es {df_income.shape[1]} ")

El número de filas es 27790 y el número de columnas es 11 


### Variables del conjunto de datos

Se revisa el listado de columnas del dataset para identificar las variables disponibles.

In [23]:
df_income.columns

Index(['STATE', 'ZIPCODE', 'Number of returns', 'Adjusted gross income (AGI)',
       'Avg AGI', 'Number of returns with total income', 'Total income amount',
       'Avg total income', 'Number of returns with taxable income',
       'Taxable income amount', 'Avg taxable income'],
      dtype='object')

### Información del dataset

Se inspecciona la estructura del dataset para confirmar que los tipos de datos son correctos y detectar valores ausentes.

In [24]:
df_income.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27790 entries, 0 to 27789
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   STATE                                  27790 non-null  object 
 1   ZIPCODE                                27790 non-null  int64  
 2   Number of returns                      27790 non-null  int64  
 3   Adjusted gross income (AGI)            27790 non-null  int64  
 4   Avg AGI                                27790 non-null  float64
 5   Number of returns with total income    27790 non-null  int64  
 6   Total income amount                    27790 non-null  int64  
 7   Avg total income                       27790 non-null  float64
 8   Number of returns with taxable income  27790 non-null  int64  
 9   Taxable income amount                  27790 non-null  int64  
 10  Avg taxable income                     27790 non-null  float64
dtypes: floa

**Observaciones:**

Se observa que el conjunto de datos no presenta valores nulos en ninguna columna, ya que todas las columnas cuentan con **27.790** registros.

Además, se verifica que las variables se encuentran en su forma adecuada según su naturaleza, con `STATE` como variable categórica y el resto de columnas en formatos numéricos coherentes para el análisis posterior.

### Registros duplicados

Se identifican los registros duplicados para evaluar la integridad del conjunto de datos.

In [25]:
df_income.duplicated().sum()

np.int64(0)

El análisis de duplicidad confirma que no existen observaciones repetidas en el conjunto de datos.

### Análisis de variables

Se revisan las variables del dataset para clasificar su tipo y evaluar su comportamiento general.

#### Variables numéricas

In [26]:
columns_num_income = df_income.select_dtypes(include='number').columns

columns_num_income

Index(['ZIPCODE', 'Number of returns', 'Adjusted gross income (AGI)',
       'Avg AGI', 'Number of returns with total income', 'Total income amount',
       'Avg total income', 'Number of returns with taxable income',
       'Taxable income amount', 'Avg taxable income'],
      dtype='object')

Se identifican 10 variables numéricas relacionadas con el código postal `ZIPCODE`, el volumen de declaraciones presentadas y diferentes métricas de ingresos.

In [27]:
df_income.describe(include='number').T

,count,mean,std,min,25%,50%,75%,max
ZIPCODE,27790.0,48853.603275,2.714044e+04,0.000000,27019.250000,48864.500000,70580.250000,9.999900e+04
Number of returns,27790.0,10346.201151,1.806534e+05,90.000000,580.000000,1900.000000,7530.000000,1.686140e+07
Adjusted gross income (AGI),27790.0,657743.318460,1.208537e+07,1482.000000,27276.000000,94466.500000,416553.750000,1.209190e+09
Avg AGI,27790.0,57.255441,4.008777e+01,8.717647,40.876167,48.539511,60.416523,1.815537e+03
Number of returns with total income,27790.0,10346.191796,1.806532e+05,90.000000,580.000000,1900.000000,7530.000000,1.686137e+07
Total income amount,27790.0,667443.110831,1.226299e+07,1503.000000,27787.500000,95842.000000,421922.500000,1.228345e+09
Avg total income,27790.0,58.240438,4.074525e+01,8.841176,41.469264,49.394989,61.642174,1.831838e+03
Number of returns with taxable income,27790.0,7882.510975,1.367683e+05,30.000000,440.000000,1420.000000,5730.000000,1.283281e+07
Taxable income amount,27790.0,451150.890176,8.310009e+06,274.000000,16545.250000,59106.000000,263607.000000,8.124731e+08
Avg taxable income,27790.0,48.970608,3.857279e+01,6.850000,33.359982,40.430130,51.449167,1.635495e+03


**Observaciones:**

El dataset cuenta con **27.790** registros y concentra variables de recuento y variables económicas.

`ZIPCODE`: aunque aparece como variable numérica, representa un código postal y, por tanto, no es una magnitud interpretable en términos de media o desviación. Además, el mínimo **0** y el máximo **99999** sugieren la presencia de códigos especiales o no válidos que conviene depurar antes de análisis.

`Number of returns` y `Number of returns with total income`: presentan estadísticos muy similares, lo que sugiere una relación estrecha entre ambas variables. Además, la diferencia entre los valores promedio y los máximos, claramente elevados, indica una distribución fuertemente asimétrica y la presencia de valores extremos.

`Adjusted gross income` y `Total income amount`: ambas variables presentan un patrón muy similar, la media es considerablemente inferior a los valores máximos, lo que indica una distribución con asimetría positiva y la existencia de unos pocos códigos postales que concentran importes extremadamente elevados frente al resto.

`Avg AGI`, `Avg total income` y `Avg taxable income`: los promedios se sitúan en rangos moderados, valores alrededor de **50**–**60**, pero los máximos superan ampliamente esos niveles llegando hasta **1.800**, lo que sugiere outliers o ZIPs con valores medios excepcionalmente altos.

`Number of returns with taxable income` y `Taxable income amount`: mantienen un patrón claramente asimétrico, con valores máximos muy elevados respecto al resto, lo que refleja una fuerte heterogeneidad entre códigos postales.

In [28]:
for col in columns_num_income:
    
    print(f"La columna {col} tiene un total de {df_income[col].nunique()} valores únicos")

    display(df_income[col].value_counts().head(10))

La columna ZIPCODE tiene un total de 27690 valores únicos


ZIPCODE
99999    51
0        51
35010     1
35014     1
35016     1
35019     1
35020     1
82836     1
35054     1
83014     1
Name: count, dtype: int64

La columna Number of returns tiene un total de 2857 valores únicos


Number of returns
140    207
170    203
110    198
160    198
200    196
240    192
180    192
150    191
120    173
130    172
Name: count, dtype: int64

La columna Adjusted gross income (AGI) tiene un total de 26379 valores únicos


...

Number of returns with taxable income
120    273
130    254
90     247
160    242
110    242
100    239
140    239
150    237
180    223
170    217
Name: count, dtype: int64

La columna Taxable income amount tiene un total de 25567 valores únicos


Taxable income amount
4528     5
11754    5
4882     5
5629     4
4009     4
11735    4
13683    4
13001    4
12486    4
27681    4
Name: count, dtype: int64

La columna Avg taxable income tiene un total de 27014 valores únicos


Avg taxable income
37.600000    7
28.300000    6
37.000000    6
39.900000    5
34.750000    5
38.000000    5
30.600000    4
32.375000    4
32.185714    4
31.000000    4
Name: count, dtype: int64

**Observaciones:**

El recuento de valores únicos indica que `ZIPCODE` es prácticamente único por registro, aunque aparecen valores repetidos como **0** y **99999**, lo que sugiere la presencia de códigos que deben depurarse.

Las variables de importes y promedios de ingresos tienen un número elevado de valores distintos, lo que sugiere un nivel de detalle alto en las magnitudes económicas por **ZIP**, mientras que los campos de número de declaraciones presentan menos variación y menos valores repetidos con frecuencia, propios de conteos.

In [29]:
df_income.median(numeric_only=True).round(2)

ZIPCODE                                  48864.50
Number of returns                         1900.00
Adjusted gross income (AGI)              94466.50
Avg AGI                                     48.54
Number of returns with total income       1900.00
Total income amount                      95842.00
Avg total income                            49.39
Number of returns with taxable income     1420.00
Taxable income amount                    59106.00
Avg taxable income                          40.43
dtype: float64

**Observaciones:**

En este caso `Number of returns` y `Number of returns with total income` comparten una mediana de **1.900**, lo que indica un volumen similar de declaraciones y declaraciones con ingresos totales reportados por ZIP. En cambio, `Number of returns with taxable income` presenta una mediana de **1.420**, lo que sugiere que no todas las declaraciones estan sujetas a tributación.

En cuanto a los importes agregados, la mediana de `Adjusted gross income` se sitúa en **94.466,5**, mientras que `Total income amount` es **95.842**, mientras `Taxable income amount` es inferior alcanzando **59.106**, este patrón es coherente con que el ingreso imponible represente una fracción del ingreso total una vez aplicados ajustes y deducciones.

Para finalizar, las variables `Avg AGI`, `Avg total income` y `Avg taxable income` presentan medianas en torno a **40**–**50**, valores consistentes con magnitudes medias por declaración según la escala del dataset, y resultan útiles para comparaciones entre códigos postales sin depender del volumen de declaraciones.

### Variables categoricas

In [30]:
columns_cate_income = df_income.select_dtypes(include=['category', 'object']).columns

columns_cate_income

Index(['STATE'], dtype='object')

Se identifica una única variable categórica en el dataset, `STATE`, que indica el estado asociado a cada código postal.

In [31]:
df_income.describe(include=['category', 'object']).T

,count,unique,top,freq
STATE,27790,51,TX,1625


**Observaciones:**

El análisis de la variable categórica `STATE` indica que el dataset incluye 51 estados distintos. El estado más frecuente es TX, con 1.625 registros, lo que sugiere una mayor representación de transacciones asociadas a ese estado dentro del conjunto de datos.

In [32]:
for col in columns_cate_income:

    print(f"Los columna {col} tiene un total de {df_income[col].nunique()} valores únicos")

    display(df_income[col].value_counts().head(10))

Los columna STATE tiene un total de 51 valores únicos


STATE
TX    1625
NY    1542
CA    1484
PA    1367
IL    1230
OH     998
FL     918
MI     892
MO     888
IA     828
Name: count, dtype: int64

**Observaciones:**

El análisis de valores únicos para `STATE` confirma que el dataset cubre 51 categorías distintas a nivel estatal, con **TX** como el estado más frecuente. 

Este resultado indica una distribución geográfica amplia, aunque con cierta concentración en algunos estados, por lo que `STATE` resulta una variable adecuada para segmentar y comparar los indicadores fiscales por territorio.